# Reporte de producción mensual

In [1]:
import streamlit as st
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def format_timedelta_hh_mm_ss(td: timedelta) -> str:
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

config = {
  'user': 'operario',
  'password': '',
  'host': '192.168.0.12', # O la IP de tu servidor
  'database': 'dbp8100'
}

def conectar_mysql():
    return mysql.connector.connect(**config)
    
def obtener_df(fecha_inicio, fecha_fin):
    db = conectar_mysql()
    cur = db.cursor()
    cur.execute('''
                SELECT
                    subconsulta.Codigo,
                    subconsulta.Nombre,
                    SUM(subconsulta.Dosificado) AS Total
                FROM
                    (   SELECT
                            tareaseje.NroID,
                            tareaseje.Fecha AS Fecha,
                            tareaseje.Hora AS Hora,
                            formulas.NroID AS IDF,
                            formulas.Codigo AS Codigo,
                            formulas.Nombre AS Nombre,
                            tareaseje.Set AS Programado,
                            (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                            tareaseje.Tiempo AS Tiempo
                        FROM
                            dbp8100.tareaseje AS tareaseje
                        JOIN
                            dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
                        WHERE
                            (tareaseje.Fecha = %s AND tareaseje.Hora >= '22:00:00') OR
                            (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '22:00:00')
                    ) AS subconsulta
                -- Agrupamos por el nombre de la fórmula para sumar los valores dosificados
                GROUP BY
                    subconsulta.Nombre;
                ''',(fecha_inicio, fecha_inicio, fecha_fin))

    resultados = cur.fetchall()
    # Obtener los nombres de las columnas
    columnas = [desc[0] for desc in cur.description]

    # Crear un DataFrame de pandas
    df = pd.DataFrame(resultados, columns=columnas)

    return df

In [3]:
df = obtener_df('2025-06-30', '2025-07-31')
df

,Codigo,Nombre,Total
0,FP0027,(LA MALVINA) DESARROLLO TERNEROS MAIZ SIN SAL,15017.00
1,FP0012,CABAÑA PRESENTACIÓN (COMAGRO),16082.50
2,FP0012*,CABAÑA PRESENTACIÓN (LA MALVINA),5984.60
3,FC820,CERDAS GESTACIÓN (LA MALVINA) MAXIMIX.,11059.75
4,FC913,CERDAS LACTANCIA (MAXIMIX) LA MALVINA..,6006.60
5,09-141,CRE. MELAZA 10% A.A. (Los Roanos),72675.65
6,09-143,CREC. MELAZA 12 % A.A....,814070.45
7,CP-001,CRECIMIENTO POWER,5125.25
8,FC633,DESARROLLO (LA MALVINA)...,20053.30
9,07-001,DESTETE PRECOZ MELAZA 10% A.A.,10072.00


In [22]:
df.to_excel('07.xlsx', index=False)